# Physics-Informed Neural Network

Neste notebook resolveremos a equação de Burgers em duas dimensões usando uma rede neural informada por física

Consulte os slides da aula 7 para a teoria e a definição do problema em mais detalhes

#Importar bibliotecas

Nesse caso, usaremos o PyTorch para poder utilizar a funcionalidade de diferenciação automática

In [ ]:
import torch
import torch.nn as nn
import numpy as np

import plotly.express as px
import plotly.graph_objects as go

np.random.seed(1)

# Geração do domínio

Nesta seção, geraremos os pontos que a rede usará como referência para encontrar a solução

## Criar função que gera pontos no contorno do domínio

Primeiro, vamos gerar pontos ao longo da condição de contorno. Nestes pontos, sabemos qual deve ser o valor de z. Eles servem para "ancorar" o resto da superfície da solução.

O domínio tem 4 lados, cada um deles terá 1/4 do número total de pontos.

In [30]:
def gerar_pontos_contorno(pontos_no_contorno,comprimento_x,tempo_final):
  pontos_por_lado = pontos_no_contorno//4

  # Lado 1 (x = 0, qualquer t)
  x_lado1 = 0 * np.ones((pontos_por_lado,1))
  t_lado1 = np.random.uniform(size=(pontos_por_lado,1),low=0,high=tempo_final)

  u_lado1 = 0 * np.ones((pontos_por_lado,1))

  # Lado 2 (x = comprimento_x, qualquer t)
  x_lado2 = comprimento_x * np.ones((pontos_por_lado,1))
  t_lado2 = np.random.uniform(size=(pontos_por_lado,1),low=0,high=tempo_final)

  u_lado2 = 0 * np.ones((pontos_por_lado,1))

  # Condicao inicial (x = qualquer, t = 0)
  x_inicial = np.random.uniform(size=(2*pontos_por_lado,1),low=0,high=comprimento_x)
  t_inicial = 0 * np.ones((2*pontos_por_lado,1))

  u_inicial = np.sin(2*np.pi*x_inicial/comprimento_x)
  #u_inicial = (1-x_inicial)*x_inicial

  # Juntar todos os lados
  x_todos = np.vstack((x_lado1,x_lado2,x_inicial))
  t_todos = np.vstack((t_lado1,t_lado2,t_inicial))
  u_todos = np.vstack((u_lado1,u_lado2,u_inicial))

  # Criar arrays X e Y
  X_contorno = np.hstack((x_todos,t_todos))
  Y_contorno = u_todos

  return X_contorno, Y_contorno

## Criar função que gera pontos de avaliação da equação

Esta é a segunda classe de pontos que usaremos. Neles, não sabemos a solução. Mas sabemos qual equação eles devem obedecer.

Estes pontos estão uniformemente distribuídos ao longo do domínio.

In [31]:
def gerar_pontos_equacao(pontos_no_dominio,comprimento_x,tempo_final):
  x_dominio = np.random.uniform(size=(pontos_no_dominio,1),low=0,high=comprimento_x)
  t_dominio = np.random.uniform(size=(pontos_no_dominio,1),low=0,high=tempo_final)

  X_equacao = np.hstack((x_dominio,t_dominio))

  return X_equacao

## Gerar pontos e plotar

Agora, vamos usar as funções que montamos acima para gerar os pontos

In [32]:
comprimento_x = 1
tempo_final = 1

pontos_no_contorno = 600
pontos_no_dominio = 1000

X_contorno, Y_contorno = gerar_pontos_contorno(pontos_no_contorno,comprimento_x,tempo_final)
X_equacao = gerar_pontos_equacao(pontos_no_dominio,comprimento_x,tempo_final)


Plotamos uma vista superior do domínio para ver se os pontos estão nos lugares corretos

In [33]:
# Vista superior
#fig = plt.figure()
#ax = fig.add_subplot()
scatter_contorno = px.scatter(x=X_contorno[:,0],y=X_contorno[:,1])
scatter_equacao = px.scatter(x=X_equacao[:,0],y=X_equacao[:,1], color_discrete_sequence=['red'])
fig = go.Figure(data=scatter_contorno.data+scatter_equacao.data)
fig.update_layout(xaxis_title='x',yaxis_title='t')
fig.show()

E uma vista em perspectiva apenas dos pontos do contorno, para ver se parece correto

In [34]:
# Vista em perspectiva
scatter_3d = px.scatter_3d(x=X_contorno[:,0].flatten(),y=X_contorno[:,1].flatten(),z=Y_contorno.flatten())
fig = go.Figure(scatter_3d)
fig.update_layout(scene=dict(aspectratio=dict(x=1.5, y=1.5, z=0.5)))
fig.show()

# Definição da Rede Neural

Neste seção, vamos definir a rede neural. Primeiro, montamos sua estrutura, depois definimos a função de perda e o otimizador.

## Estrutura da rede neural

Vamos criar uma função que monta uma rede neural totalmente conectada baseada em uma lista com o número de neurônios em cada camada.

Note que removemos a função de ativação da última camada pois estamos fazendo um ajuste de função

In [7]:
def criar_rede_neural(numero_de_neuronios):

  # Criar uma lista de todas as camadas
  camadas = []

  # Para cada camada, adicionar as conexões e a função de ativação
  for i in range(len(numero_de_neuronios)-1):
    camadas.append(nn.Linear(numero_de_neuronios[i],numero_de_neuronios[i+1]))
    camadas.append(nn.Tanh())

  # Remover a última camada, pois é a função de ativação
  camadas.pop()
  #camadas.pop()

  # Criar rede
  return nn.Sequential(*camadas)

Agora, definimos o número de neurônios por camada e chamamos a função para inicializar a rede.

Note que a primeira camada deve ter dois neurônios, pois a função tem duas entradas.

De forma similar, a última camada deve ter apenas um neurônio, pois a função tem apenas uma saída.

In [8]:
numero_de_neuronios = [2, 20, 20, 20, 1]

rna = criar_rede_neural(numero_de_neuronios)

print(rna)

Sequential(
  (0): Linear(in_features=2, out_features=20, bias=True)
  (1): Tanh()
  (2): Linear(in_features=20, out_features=20, bias=True)
  (3): Tanh()
  (4): Linear(in_features=20, out_features=20, bias=True)
  (5): Tanh()
  (6): Linear(in_features=20, out_features=1, bias=True)
)


## Definição das funções de perda

Nesta forma de implementar um PINN, precisamos de duas funções de perda diferentes. Uma será responsável pelas condições de contorno. A outra, pela observância das equações.

A perda responsável pelas condições de contorno funciona de maneira bem similar a uma rede neural convencional:
Comparamos a previsão da rede em cada ponto com o valor que temos de referência para estes pontos.

In [9]:
def calc_perda_contorno(rna,X_contorno,Y_contorno):
  Y_predito = rna(X_contorno)
  return nn.functional.mse_loss(Y_predito, Y_contorno)

Já a perda responsável pela observância das equações funciona de forma bem diferente:

Usaremos a funcionalidade de derivadas automáticas (autograd) para obter as derivadas parciais das saídas da rede em função das entradas. Essas derivadas parciais serão usadas para se calcular o resíduo da equação em cada um dos pontos.

In [10]:
def calc_residuo(rna,X_equacao):
  x = X_equacao[:,0].reshape(-1, 1)
  t = X_equacao[:,1].reshape(-1, 1)

  u = rna(torch.hstack((x,t)))

  u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u),retain_graph=True, create_graph=True)[0]
  u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u),retain_graph=True, create_graph=True)[0]
  u_t = torch.autograd.grad(u, t, grad_outputs=torch.ones_like(u),retain_graph=True, create_graph=True)[0]

  return u_t + u*u_x - 0.01/np.pi*u_xx

In [11]:
def calc_perda_equacao(rna,X_equacao):

  residuo = calc_residuo(rna,X_equacao)

  return torch.mean(torch.square(residuo))

In [12]:
def calc_perda(rna,X_contorno,Y_contorno,X_equacao,alpha=0.2):

  perda_contorno = calc_perda_contorno(rna,X_contorno,Y_contorno)
  perda_equacao = calc_perda_equacao(rna,X_equacao)

  perda = (1-alpha)*perda_contorno + alpha*perda_equacao

  return perda, perda_contorno, perda_equacao

## Definir otimizador

No PyTorch, uma das entradas para o otimizador são os parâmetros da rede. É a partir daqui que o otimizador "sabe" quais variáveis ele pode alterar.

O Agendador serve para alterar parâmetros do otimizador ao longo da execução. Neste exemplo, iremos diminuir a taxa de aprendizado. Ela será multiplicada por 0.9 a cada 1000 épocas.

O valor de alpha serve para equilibrar as perdas de contorno com as perdas de equação.

In [13]:
otimizador = torch.optim.Adam(rna.parameters(),lr=0.01)
agendador = torch.optim.lr_scheduler.StepLR(otimizador, step_size=1000, gamma=0.9)
alpha = 0.1

# Criar tensores e transferir para GPU

No PyTorch, as variáveis devem ser armazenadas em tensores.

Além disso, se formos rodar em GPU, precisamos manualmente carregar a rede a as variáveis na memória do GPU.

Note a opção "requires_grad" que está ativa na variável X_equação, isso sinaliza que o PyTorch deverá manter a trilha de todas as operações feitas a partir desta variável para, depois, conseguir usar a regra da cadeia e calcular as derivadas.



In [18]:
X_equacao = torch.tensor(X_equacao,requires_grad=True,dtype=torch.float)
X_contorno = torch.tensor(X_contorno,dtype=torch.float)
Y_contorno = torch.tensor(Y_contorno,dtype=torch.float)

device = torch.device('cuda' if torch.cuda.is_available () else 'cpu')
X_equacao = X_equacao.to(device)
X_contorno = X_contorno.to(device)
Y_contorno = Y_contorno.to(device)
rna = rna.to(device)

C:\Users\Pena\AppData\Local\Temp\ipykernel_22332\1532747031.py:1: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).

C:\Users\Pena\AppData\Local\Temp\ipykernel_22332\1532747031.py:2: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).

C:\Users\Pena\AppData\Local\Temp\ipykernel_22332\1532747031.py:3: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).



# Testar modelo

Neste seção, vamos varificar se o modelo foi construido corretamente e preparar funções para ver os resulados

## Rodar alguns passos de otimização

Primeiro, vamos rodar 10 épocas

In [19]:
# Colocar rede em modo de treinamento
rna.train()

# FAZER ITERAÇÃO
for epoca in range(10):

  # Inicializar gradientes
  otimizador.zero_grad()

  # Calcular perdas
  perda, perda_contorno, perda_equacao = calc_perda(rna,X_contorno,Y_contorno,X_equacao,alpha=alpha)

  # Backpropagation
  perda.backward()

  # Passo do otimizador
  otimizador.step()
  agendador.step()

  # Mostrar resultados
  print(f'Epoca: {epoca}, Perda: {perda.item()} (Contorno: {perda_contorno.item()}, Equacao: {perda_equacao.item()})')

Epoca: 0, Perda: 0.2460540384054184 (Contorno: 0.2733524441719055, Equacao: 0.00036855615326203406)
Epoca: 1, Perda: 0.22437967360019684 (Contorno: 0.24885493516921997, Equacao: 0.004102366976439953)
Epoca: 2, Perda: 0.21301674842834473 (Contorno: 0.23591174185276031, Equacao: 0.006961835082620382)
Epoca: 3, Perda: 0.19888973236083984 (Contorno: 0.2196287214756012, Equacao: 0.012238916009664536)
Epoca: 4, Perda: 0.1969834566116333 (Contorno: 0.21642343699932098, Equacao: 0.02202366292476654)
Epoca: 5, Perda: 0.19595755636692047 (Contorno: 0.21452662348747253, Equacao: 0.02883601002395153)
Epoca: 6, Perda: 0.1925840973854065 (Contorno: 0.21087650954723358, Equacao: 0.027952441945672035)
Epoca: 7, Perda: 0.19431407749652863 (Contorno: 0.21322965621948242, Equacao: 0.024073811247944832)
Epoca: 8, Perda: 0.19378304481506348 (Contorno: 0.21294081211090088, Equacao: 0.021363159641623497)
Epoca: 9, Perda: 0.1889444887638092 (Contorno: 0.20797684788703918, Equacao: 0.017653286457061768)


## Exibir resultados

Agora, vamos preparar uma função que calcula a rna em um grid e outra que plota a solução

In [20]:
def calcular_grid(rna, comprimento_x, tempo_final, nx=101, nt=101):

    # Definir grid
    x = np.linspace(0.,comprimento_x,nx)
    t = np.linspace(0.,tempo_final,nt)
    [t_grid, x_grid] = np.meshgrid(t,x)
    x = torch.tensor(x_grid.flatten()[:,None],requires_grad=True,dtype=torch.float).to(device)
    t = torch.tensor(t_grid.flatten()[:,None],requires_grad=True,dtype=torch.float).to(device)

    # Avaliar modelor
    rna.eval()
    Y_pred = rna(torch.hstack((x,t)))

    # Formatar resultados em array
    u_pred = Y_pred.cpu().detach().numpy()[:,0].reshape(x_grid.shape)

    return x_grid, t_grid, u_pred

In [21]:
# Calcular valores da função e gerar grids
x_grid, t_grid, u_pred = calcular_grid(rna, comprimento_x, tempo_final)

In [22]:
numero_de_epocas = 20000
perda_historico = np.zeros(numero_de_epocas)
perda_contorno_historico = np.zeros(numero_de_epocas)
perda_equacao_historico = np.zeros(numero_de_epocas)
epocas = np.array(range(numero_de_epocas))

# Colocar rede em modo de treinamento
rna.train()

# FAZER ITERAÇÃO
for epoca in epocas:

  # Resortear pontos
  #X_equacao = gerar_pontos_equacao(pontos_no_dominio,comprimento_x,tempo_final)
  #X_equacao = torch.tensor(X_equacao,requires_grad=True,dtype=torch.float).to(device)

  # Inicializar gradientes
  otimizador.zero_grad()

  # Calcular perdas
  perda, perda_contorno, perda_equacao = calc_perda(rna,X_contorno,Y_contorno,X_equacao,alpha=alpha)

  # Backpropagation
  perda.backward()

  # Passo do otimizador
  otimizador.step()
  agendador.step()

  # Guardar logs
  perda_historico[epoca] = perda.item()
  perda_contorno_historico[epoca] = perda_contorno.item()
  perda_equacao_historico[epoca] = perda_equacao.item()

  if epoca%500==0:
    print(f'Epoca: {epoca}, Perda: {perda.item()} (Contorno: {perda_contorno.item()}, Equacao: {perda_equacao.item()})')



Epoca: 0, Perda: 0.18782314658164978 (Contorno: 0.20720414817333221, Equacao: 0.013394191861152649)
Epoca: 500, Perda: 0.01983073726296425 (Contorno: 0.007565988693386316, Equacao: 0.13021346926689148)
Epoca: 1000, Perda: 0.005228936672210693 (Contorno: 0.001479976810514927, Equacao: 0.038969576358795166)
Epoca: 1500, Perda: 0.001395725761540234 (Contorno: 0.00030819419771432877, Equacao: 0.011183509603142738)
Epoca: 2000, Perda: 0.0023503913544118404 (Contorno: 0.00023308586969505996, Equacao: 0.021406138315796852)
Epoca: 2500, Perda: 0.0008715275907889009 (Contorno: 0.0003095705760642886, Equacao: 0.005929140839725733)
Epoca: 3000, Perda: 0.00038539012894034386 (Contorno: 0.0001066048935172148, Equacao: 0.0028944569639861584)
Epoca: 3500, Perda: 0.00024877191754058003 (Contorno: 6.481733726104721e-05, Equacao: 0.0019043630454689264)
Epoca: 4000, Perda: 0.017180699855089188 (Contorno: 0.00673979427665472, Equacao: 0.11114884912967682)
Epoca: 4500, Perda: 0.004430722910910845 (Contorno

In [24]:
# Calcular valores da função e gerar grids
x_grid, y_grid, z_pred = calcular_grid(rna, comprimento_x, tempo_final)

# Plotar figura
fig = go.Figure(data=[go.Surface(x=x_grid, y=y_grid, z=z_pred)])

fig.update_layout(scene=dict(aspectratio=dict(x=1.5, y=1.5, z=0.5)))

In [38]:
# Plotar histórico
fig = go.FigureWidget()
fig.add_trace(go.Scatter(x=epocas, y=perda_historico, name='Total', line=dict(color='black', width=4)))
fig.add_trace(go.Scatter(x=epocas, y=perda_contorno_historico, name='Contorno', line=dict(color='blue', width=2)))
fig.add_trace(go.Scatter(x=epocas, y=perda_equacao_historico, name='Equacao', line=dict(color='red', width=2)))
fig.update_yaxes(type="log")
fig.show(renderer="colab")

FigureWidget({
    'data': [{'line': {'color': 'black', 'width': 4},
              'name': 'Total',
              'type': 'scatter',
              'uid': 'b50b16aa-760d-4f20-8f93-040a0a75077e',
              'x': {'bdata': ('AAABAAIAAwAEAAUABgAHAAgACQAKAA' ... '4WThdOGE4ZThpOG04cTh1OHk4fTg=='),
                    'dtype': 'i2'},
              'y': {'bdata': ('AAAAwJYKyD8AAADAJDzIPwAAAAADIM' ... 'DEGRI/AAAAoKEREj8AAACA0fYRPw=='),
                    'dtype': 'f8'}},
             {'line': {'color': 'blue', 'width': 2},
              'name': 'Contorno',
              'type': 'scatter',
              'uid': 'fc854f41-6784-43f1-a1d7-cdb18067b8b3',
              'x': {'bdata': ('AAABAAIAAwAEAAUABgAHAAgACQAKAA' ... '4WThdOGE4ZThpOG04cTh1OHk4fTg=='),
                    'dtype': 'i2'},
              'y': {'bdata': ('AAAAYKqFyj8AAACAjsbKPwAAAIAsqM' ... 'D1H+U+AAAAgIJC5D4AAAAg66TkPg=='),
                    'dtype': 'f8'}},
             {'line': {'color': 'red', 'width': 2},
              'name